# Flux to Mag And Deredden

author: Sam Schmidt

last successfully run: Dec 17, 2025

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import rail.interactive as ri
import tables_io
from rail.utils.path_utils import find_rail_file

DOWNLOADS_DIR = Path("../examples_data")
DOWNLOADS_DIR.mkdir(exist_ok=True)

In [ ]:
example_file = find_rail_file("examples_data/testdata/rubin_dm_dc2_example.pq")

# read in the data from the file
test_data = tables_io.read(example_file)

In [ ]:
test_data.info()

### Fluxes to Mags

To convert fluxes to mags, we need to specify patterns for the `flux_name` and `flux_err_name` columns to be converted, and the `mag_name` and `mag_err_name` columns that will store the newly created magnitudes.

This is done as below, by specifying a string listing the bands, and `{band}` in the patterns where the individual bands will go.  The dictionary below duplicates the default behavior of the converter, but is written out explicitly as an example:

In [ ]:
# convert "gaap" fluxes to magnitudes:
ftomagdict = dict(
    bands="ugrizy",
    flux_name="{band}_gaap1p0Flux",
    flux_err_name="{band}_gaap1p0FluxErr",
    mag_name="mag_{band}_lsst",
    mag_err_name="mag_err_{band}_lsst",
    copy_cols=dict(ra="ra", dec="decl", objectId="objectId"),
)
mags_data = ri.tools.photometry_tools.lsst_flux_to_mag_converter(
    input=test_data, **ftomagdict
)

In [ ]:
list(mags_data["output"].keys())

### Deredden Mags

To deredden magnitudes we need to grab one of the dust maps used by the `dustmaps` package.  We'll grab the default Schlegel-Finkbeiner-Davis "SFD" map.  NOTE: This will download a file to your machine containing the SFD data, which we will delete afterwards

In [ ]:
deredden_data = ri.tools.photometry_tools.dereddener(
    input=mags_data["output"], dustmap_dir=DOWNLOADS_DIR
)

In [ ]:
# remove the downloaded files, and the dustmap directory
for file in (DOWNLOADS_DIR / "sfd").iterdir():
    file.unlink()
(DOWNLOADS_DIR / "sfd").rmdir()

In [ ]:
deredden_data["output"].keys()

We see that the deredden stage returns us a dictionary with the dereddened magnitudes. Let's plot the difference of the un-dereddened magnitudes and the dereddened ones for u-band to see if they are, indeed, slightly brighter:

In [ ]:
delta_u_mag = mags_data["output"]["mag_u_lsst"] - deredden_data["output"]["mag_u_lsst"]
plt.figure(figsize=(8, 6))
plt.scatter(mags_data["output"]["mag_u_lsst"], delta_u_mag, s=15)
plt.xlabel("orignal u-band mag", fontsize=12)
plt.ylabel("u - deredden_u")